In [230]:
import sys
from pathlib import Path
from datetime import datetime
from dateutil.tz import gettz

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras import utils
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import GridSearchCV

np.random.seed(757566)

In [231]:
fname = 'private_dog0_correct_plus' # private_dog0_correct_plus
log_to_file = True

tensorboard_dir = '../logs/tensorboard'
logs_dir = '../logs'
timestamp = '{:%Y-%m-%dT%H:%M}'.format(datetime.now(gettz("Europe/London")))
logs_dir = logs_dir +'/' + timestamp
tensorboard_dir = tensorboard_dir +'/' + timestamp
if 'private' in fname:
    fdir = '../data/private_data/private_events_dev2' 
else:
    fdir = '../data' 

In [232]:
def readucr(filename):
    ''' Load a dataset from a file in UCR format
    space delimited, class labels in the first column.
    Returns
    X : DNN input data
    Y : class labels
    '''
    data = np.loadtxt(Path(filename))
    Y = data[:,0]
    X = data[:,1:]
    return X, Y

In [233]:
# Hyperparameter grid search adapted from Machine Learning Mastery
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
# Use scikit-learn to grid search the batch size and epochs

# Function to create model, will be parse to KerasClassifier
def create_fcn(input_shape=(150,1), num_features=100, filter_size=10, pooling_size=3, dropout=0.5):
    ''' Create FCN model '''
    num_features0 = num_features
    num_features1 = math.floor(1.5 * num_features)
    nb_classes = 2
    x = Input(shape=(input_shape))
    conv_x = keras.layers.Conv1D(num_features0, filter_size, activation='relu')(x)
    conv_x = keras.layers.Conv1D(num_features0, filter_size, activation='relu')(conv_x)
    conv_x = keras.layers.MaxPooling1D(pooling_size)(conv_x)
    conv_x = keras.layers.Conv1D(num_features1, filter_size, activation='relu')(conv_x)
    conv_x = keras.layers.Conv1D(num_features1, filter_size, activation='relu')(conv_x)
    full = keras.layers.GlobalAveragePooling1D()(conv_x)
    y = Dropout(dropout,name='Dropout')(full)
    out = Dense(nb_classes, activation='sigmoid')(full)
    model = Model(x, out)
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [234]:
def prepare_data(y):
    ''' Return y as a categorical array'''
    nb_classes = 2
    y = (y - y.min())/(y.max()-y.min())*(nb_classes-1)
    Y = utils.to_categorical(y, nb_classes)
    return Y

In [235]:
# load dataset
x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
x_test, y_test = readucr(fdir+'/'+fname+'/'+fname+'_TEST.txt')
X = np.concatenate((x_train, x_test), axis=0)
Y = np.concatenate((y_train, y_test), axis=0)
X = X.reshape(X.shape + (1,))
input_shape = X.shape[1:]
print(input_shape)
Y = prepare_data(Y)

# Add callbacks
callbacks = []
tb_dir = tensorboard_dir+'/'+fname
Path(tb_dir).mkdir(parents=True, exist_ok=True) 
callbacks.append(keras.callbacks.TensorBoard(log_dir=tb_dir, histogram_freq=0))

# define the grid search parameters
batch_size = 32
epochs = 1
num_features = [32] # [32, 64]
filter_size = [4] # [4, 16]
pooling_size = [4] # [4, 8]
dropout = 0.5
param_grid = dict(num_features=num_features, filter_size=filter_size, pooling_size=pooling_size)

# Create model and run the grid search
model = KerasClassifier(build_fn=create_fcn, 
                        input_shape=input_shape,
                        dropout=dropout,
                        batch_size=batch_size, epochs=epochs,
                        verbose=1)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, error_score=0) #fit_params={'callbacks': callbacks})
grid_result = grid.fit(X, Y)

# Summarise results
print('Best score:', grid_result.best_score_, 'using: ', grid_result.best_params_)
cv = pd.DataFrame(grid_result.cv_results_)
pd.set_option('display.max_colwidth', -1)
cv[['mean_test_score', 'std_test_score', 'params']]

(1000, 1)
102/102 [==============================] - 2s 22ms/sample - loss: 0.6883 - acc: 0.4804
Best score: 0.529411772886912 using:  {'pooling_size': 4, 'num_features': 32, 'filter_size': 4}


,mean_test_score,std_test_score,params
0,0.529412,0.024015,"{'pooling_size': 4, 'num_features': 32, 'filter_size': 4}"


In [236]:
grid_result.cv_results_
cv = pd.DataFrame(grid_result.cv_results_)
cv

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_filter_size,param_num_features,param_pooling_size,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,32.829951,9.503478,0.529412,0.534314,4,32,4,"{'pooling_size': 4, 'num_features': 32, 'filter_size': 4}",1,0.529412,0.529412,0.5,0.544118,0.558824,0.529412,0.401041,0.142044,0.024015,0.006932
